Scenes Classification

1. Tải bộ dữ liệu: Các bạn tải bộ dữ liệu img_cls_scenes_classification.zip trong
đường dẫn thuộc mục Datasets tại phần III (trong tập lý thuyết).
2. Import các thư viện cần thiết:

In [ ]:
import torch
import torch.nn as nn
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import train_test_split

3. Cố định giá trị ngẫu nhiên:

In [ ]:
def set_seed(seed):
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

seed = 59
set_seed(seed)

4. Đọc dữ liệu: Sau khi tải và giải nén bộ dữ liệu, chúng ta sẽ được một thư mục chứa
dữ liệu như trong file thuyết minh

So với bộ dữ liệu ở bài trước, ta đã được chia sẵn hai bộ dữ liệu train và val. Tuy
nhiên, để đồng bộ, chúng ta sẽ vẫn đi theo hướng code cũ. Đầu tiên, ta vẫn đọc danh
sách classes như sau:

In [ ]:
root_dir = 'scenes_classification '
train_dir = os.path.join(root_dir , 'train')
test_dir = os.path.join(root_dir , 'val')

classes = {
label_idx: class_name \
for label_idx , class_name in enumerate(
sorted(os.listdir(train_dir))
)
}

Tiếp đến, ta đọc lên toàn bộ các đường dẫn ảnh cũng như label tương ứng. Tuy nhiên,
ta sẽ coi thư mục val là tập test của bộ dữ liệu và sẽ khai báo danh sách riêng cho bộ
này. Như vậy, ta có code như sau:

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []

for dataset_path in [train_dir , test_dir ]:
for label_idx , class_name in classes.items():
class_dir = os.path.join(dataset_path , class_name)
for img_filename in os.listdir(class_dir):
img_path = os.path.join(class_dir , img_filename)
if 'train' in dataset_path:
X_train.append(img_path)
y_train.append(label_idx)
else:
X_test.append(img_path)
y_test.append(label_idx)

5. Chia bộ dữ liệu train, val, test: Vì đã có sẵn bộ dữ liệu train và test, ta chỉ việc
chia thêm cho tập val từ tập train như sau:

In [ ]:
seed = 0
val_size = 0.2
is_shuffle = True

X_train , X_val , y_train , y_val = train_test_split(
X_train , y_train ,
test_size=val_size ,
random_state=seed ,
shuffle=is_shuffle
)

6. Xây dựng class pytorch datasets: Tương tự ở bài trước, ta xây dựng pytorch
dataset cho bộ dữ liệu Scenes như sau:

In [ ]:
class ScenesDataset(Dataset):
def __init__(
self ,
X, y,
transform=None
):
self.transform = transform
self.img_paths = X
self.labels = y

def __len__(self):
return len(self.img_paths)

def __getitem__(self , idx):
img_path = self.img_paths[idx]
img = Image.open(img_path).convert("RGB")

if self.transform:
img = self.transform(img)

return img , self.labels[idx]

7. Xây dựng hàm tiền xử lý ảnh (transforms): Tương tự như bài trước, ta xây
dựng hàm tiền xử lý ảnh như sau:

In [ ]:
def transform(img , img_size =(224 , 224)):
img = img.resize(img_size)
img = np.array(img)[..., :3]
img = torch.tensor(img).permute(2, 0, 1).float ()
normalized_img = img / 255.0

return normalized_img

8. Khai báo datasets object cho ba bộ train, val, test: Tương tự như bài trên, ta
khai báo ba object datasets tương ứng cho ba bộ dữ liệu train, val, test như sau:

In [ ]:
train_dataset = ScenesDataset(
X_train , y_train ,
transform=transform
)
val_dataset = ScenesDataset(
X_val , y_val ,
transform=transform
)
test_dataset = ScenesDataset(
X_test , y_test ,
transform=transform
)

9. Khai báo dataloader: Lưu ý, với colab, các bạn nên cài đặt train batch size = 64
để có thể train được bài này.

In [ ]:
train_batch_size = 64
test_batch_size = 8

train_loader = DataLoader(
train_dataset ,
batch_size=train_batch_size ,
shuffle=True
)
val_loader = DataLoader(
val_dataset ,
batch_size=test_batch_size ,
shuffle=False
)
test_loader = DataLoader(
test_dataset ,
batch_size=test_batch_size ,
shuffle=False
)

10. Xây dựng model: Trong phần này, chúng ta sẽ xây dựng class cho model deep
learning với kiến trúc DenseNet. Thông tin tổng quan về kiến trúc DenseNet được thể
hiện ở bảng trong thuyết minh

Đầu tiên, chúng ta sẽ xây dựng class Dense Block, có ảnh mô phỏng như hình dưới
đây:

In [ ]:
class BottleneckBlock(nn.Module):
def __init__(self , in_channels , growth_rate):
super(BottleneckBlock , self).__init__ ()
self.bn1 = nn.BatchNorm2d(in_channels)
self.conv1 = nn.Conv2d(in_channels , 4 * growth_rate ,kernel_size =1, bias=False)
self.bn2 = nn.BatchNorm2d (4 * growth_rate)
self.conv2 = nn.Conv2d (4 * growth_rate , growth_rate ,kernel_size =3, padding=1, bias=False)
self.relu = nn.ReLU()

def forward(self , x):
res = x.clone().detach ()
x = self.bn1(x)
x = self.relu(x)
x = self.conv1(x)
x = self.bn2(x)
x = self.relu(x)
x = self.conv2(x)
x = torch.cat([res , x], 1)

return x

class DenseBlock(nn.Module):
def __init__(self , num_layers , in_channels , growth_rate):
super(DenseBlock , self).__init__ ()
layers = []
for i in range(num_layers):
layers.append(BottleneckBlock(in_channels + i *growth_rate , growth_rate))
self.block = nn.Sequential (* layers)

def forward(self , x):
    return self.block(x)

Với DenseBlock, ta triển khai toàn bộ kiến trúc DenseNet như sau:

In [ ]:
class DenseNet(nn.Module):
def __init__(self , num_blocks , growth_rate , num_classes):
super(DenseNet , self).__init__ ()
self.conv1 = nn.Conv2d(3, 2 * growth_rate , kernel_size =7,padding=3, stride=2, bias=False)
self.bn1 = nn.BatchNorm2d (2 * growth_rate)
self.pool1 = nn.MaxPool2d(kernel_size =3, stride=2, padding =1)

self.dense_blocks = nn.ModuleList ()
in_channels = 2 * growth_rate
for i, num_layers in enumerate(num_blocks):
self.dense_blocks.append(DenseBlock(num_layers ,in_channels , growth_rate))
in_channels += num_layers * growth_rate
if i != len(num_blocks) - 1:
out_channels = in_channels // 2
self.dense_blocks.append(nn.Sequential(
nn.BatchNorm2d(in_channels),
nn.Conv2d(in_channels , out_channels , kernel_size=1, bias=False),
nn.AvgPool2d(kernel_size =2, stride =2)
))
in_channels = out_channels

self.bn2 = nn.BatchNorm2d(in_channels)
self.pool2 = nn.AvgPool2d(kernel_size =7)
self.relu = nn.ReLU()
self.fc = nn.Linear(in_channels , num_classes)

def forward(self , x):
x = self.conv1(x)
x = self.bn1(x)
x = self.relu(x)
x = self.pool1(x)

for block in self.dense_blocks:
x = block(x)

x = self.bn2(x)
x = self.relu(x)
x = self.pool2(x)
x = x.view(x.size (0), -1)
x = self.fc(x)

return x

Cuối cùng, khai báo model DenseNet bằng đoạn code sau (ở đây ta sẽ sử dụng phiên
bản DenseNet-121)

In [ ]:
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available () else 'cpu'

model = DenseNet(
    [6, 12, 24, 16],
growth_rate =32,
num_classes=n_classes
).to(device)

11. Khai báo hàm loss và thuật toán huấn luyện: Với bài toán là phân loại ảnh, ta
cũng sẽ sử dụng hàm loss CrossEntropy và Stochastic Gradient:

In [ ]:
lr = 1e-2
epochs = 15

criterion = nn.CrossEntropyLoss ()
optimizer = torch.optim.SGD(
model.parameters (),
lr=lr
)

12. Thực hiện huấn luyện: Sử dụng hàm evaluate() và hàm fit() đã triển khai trong
bài trước, chúng ta sẽ huấn luyện model DenseNet như sau:

In [ ]:
train_losses , val_losses = fit(
model ,
train_loader ,
val_loader ,
criterion ,
optimizer ,
device ,
epochs
)

13. Đánh giá model: Ta gọi hàm evaluate() để đánh giá performance của model trên
hai tập val và test như sau

In [ ]:
val_loss , val_acc = evaluate(
model ,
val_loader ,
criterion ,
device
)
test_loss , test_acc = evaluate(
model ,
test_loader ,
criterion ,
device
)

print('Evaluation on val/test dataset ')
print('Val accuracy: ', val_acc)
print('Test accuracy: ', test_acc)

Triển khai mô hình: Với hai mô hình về phân loại ảnh thời tiết và phân loại ảnh phong
cảnh đã huấn luyện được ở phía trên, chúng ta có thể triển khai lên streamlit như ảnh sau
(thông tin về code cài đặt và link web streamlit được đề cập tại phần III trong thuyết minh):